In [0]:
from __future__ import print_function
from tensorflow import keras
import numpy as np

import numpy as np
import tensorflow as tf
from tensorflow.contrib import rnn
import random
import collections
import time

In [0]:
import numpy as np
from operator import itemgetter
from nltk import word_tokenize, sent_tokenize
from string import punctuation
from nltk.util import ngrams
import re
from sklearn.model_selection import train_test_split 

In [3]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [4]:
# Pre-Processing Dataset -Filtering out symbols, newlines. 
#Parsing and dividing in 80-20 ratio


# DATASET- Jane Austen Novels: The Complete Works of Jane Austen 
#take one line, neglect empty line and then add it to line_set then lower down whole set(string)
with open("JaneAusten.txt", 'rt') as source_file:  
    line_set = ['']
    for line in (l.rstrip() for l in source_file):
        if line != '' or line_set[-1] != '\n':                 
            line_set.append(line + '\n')
    text = "".join(line_set)
text = text.lower()  

# Tokenizing sentences from processed text
vocabulary = {}#initilaizing vocab dict
corpus = []#intializing corpus
sentence_set = sent_tokenize(text)

for sentence in sentence_set: #taking one sentence from sentence list
    words = word_tokenize(sentence) #work tokenizing each word of that sentence
    processed_sentence = ['<s>'] #making processed sentence out of it and adding START <s> symbol 
    for word in words:
        if len(set(word).intersection(punctuation)) == 0 and (len(word)>1 or word == 'a' or word == 'i') :
            processed_sentence.append(word)#adding processed and selected word to my new processed sentence
            if word in vocabulary.keys():#adding in vobaulary list if not exist else increasing frequecny by 1 if exists
                vocabulary[word] += 1
            else:
                vocabulary[word] = 1
                
    processed_sentence.append('</s>')#adding END </s> to the end of my sentence
    corpus.append(" ".join(processed_sentence))#adding processed sentence to the corpus

#Now Processed Corpus is ready    
number_of_sentence = len(corpus)
# Vocabulary list doesn't contain the START and END word used in Pre-Processing
vocabulary['<s>'] = 2*number_of_sentence
vocabulary['</s>'] = 2*number_of_sentence
# Now we have Processed sentence corpus and vocabulary list. 
#Processed Corpus DATA ANALYSIS  
tokens = sum(vocabulary.values())
types = len(vocabulary)
print ('Number of Sentences = ' + str(number_of_sentence))
print ('Number of Tokens = ' + str(tokens))
print ('Number of Types = ' + str(types))
print ("TTR= " + str(types/tokens))
# train-test split in 80:20 ratio
train_data, test_data = train_test_split(corpus, test_size=0.20, random_state=42)

Number of Sentences = 35056
Number of Tokens = 914604
Number of Types = 14651
TTR= 0.01601895465141198


In [0]:
 # tf.reset_default_graph()

start_time = time.time()
def elapsed(sec):
    if sec<60:
        return str(sec) + " sec"
    elif sec<(60*60):
        return str(sec/60) + " min"
    else:
        return str(sec/(60*60)) + " hr"

def read_data(content):
    content = [x.strip() for x in content]
    content = [word for i in range(len(content)) for word in content[i].split()]
    content = np.array(content)
    return content

# new_train = []
# for i in train_text:
#   new = ''.join(i.split('<UNK>'))
#   new_train.append(new)

In [6]:


training_data = read_data(train_data)
testing_data = read_data(test_data)
print("Loaded training data...")

def build_dataset(words):
    count = collections.Counter(words).most_common()
    dictionary = dict()
    for word, _ in count:
        dictionary[word] = len(dictionary)
    reverse_dictionary = dict(zip(dictionary.values(), dictionary.keys()))
    return dictionary, reverse_dictionary

dictionary, reverse_dictionary = build_dataset(training_data)
vocab_size = len(dictionary)


Loaded training data...


In [0]:
# A dictionary mapping words to an integer index
word_index = dictionary.copy()

# The first indices are reserved
word_index = {k:(v+3) for k,v in word_index.items()}
word_index["<PAD>"] = 0
# word_index["<START>"] = 1
word_index["<UNK>"] = 1  # unknown
word_index["<UNUSED>"] = 2

reverse_word_index = dict([(value, key) for (key, value) in word_index.items()])

def decode_review(text):
    return ' '.join([reverse_word_index.get(i, '?') for i in text])

train_text = []
for i in train_data:
  train_text.append(decode_review(i))

In [0]:
# from tensorflow.python.client import device_lib
# device_lib.list_local_devices()

In [0]:
import warnings
warnings.filterwarnings("ignore")

In [10]:
# Parameters
learning_rate = 0.0001
# training_iters = 10 * len(training_data) #10000
training_iters = 20000
display_step = 1000
n_input = 4


logs_path = '/tmp/tensorflow/rnn_words'
writer = tf.summary.FileWriter(logs_path)

# number of units in RNN cell
n_hidden = 128

# tf Graph input
x = tf.placeholder("float", [None, n_input, 1])
y = tf.placeholder("float", [None, vocab_size])

# RNN output node weights and biases
weights = {
    'out': tf.Variable(tf.random_normal([n_hidden, vocab_size]))
}
biases = {
    'out': tf.Variable(tf.random_normal([vocab_size]))
}

def RNN(x, weights, biases):

    # reshape to [1, n_input]
    x = tf.reshape(x, [-1, n_input])

    # Generate a n_input-element sequence of inputs
    # (eg. [had] [a] [general] -> [20] [6] [33])
    x = tf.split(x,n_input,1)

    # 2-layer LSTM, each layer has n_hidden units.
    #rnn_cell = rnn.MultiRNNCell([rnn.LSTMCell(n_hidden, reuse =tf.AUTO_REUSE),rnn.LSTMCell(n_hidden, reuse =tf.AUTO_REUSE)])

    # 1-layer LSTM with n_hidden units 
    #rnn_cell = rnn.LSTMCell(n_hidden, reuse =tf.AUTO_REUSE)
    
    # 1-layer RNN with n_hidden units 
    #rnn_cell = rnn.MultiRNNCell([rnn.GRUCell(n_hidden), rnn.GRUCell(n_hidden)])
    rnn_cell = rnn.BasicRNNCell(n_hidden)
    # rnns.append(rnn.BasicRNNCell(n_hidden))

    # generate prediction
    # outputs, states = rnn.static_rnn(rnn_cell, x, dtype=tf.float32)
    outputs, states = rnn.static_rnn(rnn_cell, x, dtype=tf.float32)

    # there are n_input outputs but
    # we only want the last output
    return tf.matmul(outputs[-1], weights['out']) + biases['out']

pred = RNN(x, weights, biases)

# Loss and optimizer
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=pred, labels=y))
optimizer = tf.train.RMSPropOptimizer(learning_rate=learning_rate).minimize(cost)

# Model evaluation
correct_pred = tf.equal(tf.argmax(pred,1), tf.argmax(y,1))
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

# Initializing the variables
init = tf.global_variables_initializer()

# Launch the graph
with tf.Session() as session:
    session.run(init)
    step = 0
    offset = random.randint(0,n_input+1)
    end_offset = n_input + 1
    acc_total = 0
    loss_total = 0
    loss_overall = 0
    writer.add_graph(session.graph)

    while step < training_iters:
        # Generate a minibatch. Add some randomness on selection process.
        if offset > (len(training_data)-end_offset):
            offset = random.randint(0, n_input+1)

        symbols_in_keys = [ [dictionary[ str(training_data[i])]] for i in range(offset, offset+n_input) ]
        symbols_in_keys = list(np.reshape(np.array(symbols_in_keys), [-1, n_input, 1]))

        symbols_out_onehot = np.zeros([vocab_size], dtype=float)
        symbols_out_onehot[dictionary[str(training_data[offset+n_input])]] = 1.0
        symbols_out_onehot = np.reshape(symbols_out_onehot,[1,-1])

        _, acc, loss, onehot_pred = session.run([optimizer, accuracy, cost, pred], \
                                                feed_dict={x: symbols_in_keys, y: symbols_out_onehot})
        loss_total += loss
        loss_overall += loss
        acc_total += acc
        if (step+1) % display_step == 0:
            print("No.= " + str((step+1)//display_step) + ", Average Loss= " + \
                  "{:.6f}".format(loss_total/display_step) + ", Average Accuracy= " + \
                  "{:.2f}%".format(100*acc_total/display_step))
            acc_total = 0
            loss_total = 0
            symbols_in = [training_data[i] for i in range(offset, offset + n_input)]
            symbols_out = training_data[offset + n_input]
            symbols_out_pred = reverse_dictionary[int(tf.argmax(onehot_pred, 1).eval())]
            print("%s - [%s] vs [%s]" % (symbols_in,symbols_out,symbols_out_pred))
        step += 1
        offset += (n_input+1)
    print("Optimization Finished!")
    print("Elapsed time: ", elapsed(time.time() - start_time))
    


    num_start = 0
    n_input = 4
    words = [word_index['<PAD>']] * (n_input-1) + ['<s>']
    overall = list(words)
    sentence = ''
    num_words = 0
    while num_start < n_input and num_words < 500:
        try:
            symbols_in_keys = [dictionary[str(words[i])] for i in range(len(words))]
            keys = np.reshape(np.array(symbols_in_keys), [-1, n_input, 1])
            onehot_pred = session.run(pred, feed_dict={x: keys})
            onehot_pred_index = int(tf.argmax(onehot_pred, 1).eval())
            if onehot_pred_index == dictionary['</s>']:
                sentence += ". "
                num_start += 1
            else:
                sentence = "%s %s" % (sentence,reverse_dictionary[onehot_pred_index])
            symbols_in_keys = list(symbols_in_keys[1:])
            symbols_in_keys.append(onehot_pred_index)
        except:
            onehot_pred_index = dictionary['the']
            sentence = "%s %s" % (sentence,reverse_dictionary[onehot_pred_index])
            symbols_in_keys = list(symbols_in_keys[1:])
            symbols_in_keys.append(onehot_pred_index)
        num_words += 1
        

    print(sentence)

Instructions for updating:
This class is equivalent as tf.keras.layers.SimpleRNNCell, and will be replaced by that in Tensorflow 2.0.
Instructions for updating:
Please use `keras.layers.RNN(cell, unroll=True)`, which is equivalent to this API
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See `tf.nn.softmax_cross_entropy_with_logits_v2`.

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
No.= 1, Average Loss= 30.414194, Average Accuracy= 0.40%
['fear', 'that', 'you', 'will'] - [be] vs [imposition]
No.= 2, Average Loss= 24.301142, Average Accuracy= 0.20%
['forgive', 'he', 'is', 'a'] - [fortunate] 

In [0]:
train_perplexity = tf.exp(loss)

In [12]:
with tf.Session() as session:
  print(session.run(train_perplexity))

110084.34


In [13]:
# Parameters
# learning_rate = 0.0001
# # training_iters = 10 * len(training_data) #10000
# training_iters = 100
# display_step = 10
# n_input = 4



# number of units in RNN cell
# n_hidden = 128

# tf Graph input
x = tf.placeholder("float", [None, n_input, 1])
y = tf.placeholder("float", [None, vocab_size])

# RNN output node weights and biases
weights = {
    'out': tf.Variable(tf.random_normal([n_hidden, vocab_size]))
}
biases = {
    'out': tf.Variable(tf.random_normal([vocab_size]))
}

def RNN(x, weights, biases):

    # reshape to [1, n_input]
    x = tf.reshape(x, [-1, n_input])

    # Generate a n_input-element sequence of inputs
    # (eg. [had] [a] [general] -> [20] [6] [33])
    x = tf.split(x,n_input,1)

    # 1-layer LSTM with n_hidden units 
    rnn_cell = rnn.LSTMCell(n_hidden, reuse =tf.AUTO_REUSE)

    outputs, states = rnn.static_rnn(rnn_cell, x, dtype=tf.float32)

    # there are n_input outputs but
    # we only want the last output
    return tf.matmul(outputs[-1], weights['out']) + biases['out']

pred = RNN(x, weights, biases)

# Loss and optimizer
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=pred, labels=y))
optimizer = tf.train.RMSPropOptimizer(learning_rate=learning_rate).minimize(cost)

# Model evaluation
correct_pred = tf.equal(tf.argmax(pred,1), tf.argmax(y,1))
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

# Initializing the variables
init = tf.global_variables_initializer()

# Launch the graph
with tf.Session() as session:
    session.run(init)
    step = 0
    offset = random.randint(0,n_input+1)
    end_offset = n_input + 1
    acc_total = 0
    loss_total = 0
    loss_overall = 0
    writer.add_graph(session.graph)

    while step < training_iters:
        # Generate a minibatch. Add some randomness on selection process.
        if offset > (len(training_data)-end_offset):
            offset = random.randint(0, n_input+1)

        symbols_in_keys = [ [dictionary[ str(training_data[i])]] for i in range(offset, offset+n_input) ]
        symbols_in_keys = list(np.reshape(np.array(symbols_in_keys), [-1, n_input, 1]))

        symbols_out_onehot = np.zeros([vocab_size], dtype=float)
        symbols_out_onehot[dictionary[str(training_data[offset+n_input])]] = 1.0
        symbols_out_onehot = np.reshape(symbols_out_onehot,[1,-1])

        _, acc, loss, onehot_pred = session.run([optimizer, accuracy, cost, pred], \
                                                feed_dict={x: symbols_in_keys, y: symbols_out_onehot})
        loss_total += loss
        loss_overall += loss
        acc_total += acc
        if (step+1) % display_step == 0:
            print("No.= " + str((step+1)//display_step) + ", Average Loss= " + \
                  "{:.6f}".format(loss_total/display_step) + ", Average Accuracy= " + \
                  "{:.2f}%".format(100*acc_total/display_step))
            acc_total = 0
            loss_total = 0
            symbols_in = [training_data[i] for i in range(offset, offset + n_input)]
            symbols_out = training_data[offset + n_input]
            symbols_out_pred = reverse_dictionary[int(tf.argmax(onehot_pred, 1).eval())]
            print("%s - [%s] vs [%s]" % (symbols_in,symbols_out,symbols_out_pred))
        step += 1
        offset += (n_input+1)
    print("Optimization Finished!")
    print("Elapsed time: ", elapsed(time.time() - start_time))
    


    num_start = 0
    # n_input = 4
    words = [word_index['<PAD>']] * (n_input-1) + ['<s>']
    overall = list(words)
    sentence = ''
    num_words = 0
    while num_start < 5 and num_words < 500:
        try:
            symbols_in_keys = [dictionary[str(words[i])] for i in range(len(words))]
            keys = np.reshape(np.array(symbols_in_keys), [-1, n_input, 1])
            onehot_pred = session.run(pred, feed_dict={x: keys})
            onehot_pred_index = int(tf.argmax(onehot_pred, 1).eval())
            if onehot_pred_index == dictionary['</s>']:
                sentence += ". "
                num_start += 1
            else:
                sentence = "%s %s" % (sentence,reverse_dictionary[onehot_pred_index])
            symbols_in_keys = list(symbols_in_keys[1:])
            symbols_in_keys.append(onehot_pred_index)
        except:
            onehot_pred_index = dictionary['the']
            sentence = "%s %s" % (sentence,reverse_dictionary[onehot_pred_index])
            symbols_in_keys = list(symbols_in_keys[1:])
            symbols_in_keys.append(onehot_pred_index)
        num_words += 1
        

    print(sentence)

Instructions for updating:
This class is equivalent as tf.keras.layers.LSTMCell, and will be replaced by that in Tensorflow 2.0.
No.= 1, Average Loss= 14.133432, Average Accuracy= 0.00%
['some', 'reason', 'to', 'fear'] - [that] vs [costs]
No.= 2, Average Loss= 10.907668, Average Accuracy= 1.70%
['all', 'delighted', 'to', 'forgive'] - [he] vs [excursions]
No.= 3, Average Loss= 9.728909, Average Accuracy= 5.40%
['enough', 'to', 'support', 'me'] - [in] vs [the]
No.= 4, Average Loss= 9.164069, Average Accuracy= 6.40%
['rudely', 'and', 'coarse', '</s>'] - [<s>] vs [<s>]
No.= 5, Average Loss= 8.671674, Average Accuracy= 7.10%
['which', 'appeared', 'perfectly', 'natural'] - [as] vs [</s>]
No.= 6, Average Loss= 8.217985, Average Accuracy= 8.90%
['feelings', 'you', 'see', 'did'] - [not] vs [the]
No.= 7, Average Loss= 8.225172, Average Accuracy= 9.40%
['to', 'hope', 'that', 'she'] - [tell] vs [a]
No.= 8, Average Loss= 7.744772, Average Accuracy= 8.50%
['she', 'ought', 'to', 'feel'] - [none] vs [

In [0]:
train_perplexity = tf.exp(loss)

In [0]:
with tf.Session() as session:
  print(session.run(train_perplexity))

21526436.0
